# Introduction #
This notebook is being used for the development of a data classifier for CalPolyDnD's capstone project. The classifier needs to be able to determine what kind of data is in each column of a dataset.

In [183]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import LinearSVC

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

import joblib as jl

### Data Transformation and Cleaning ###

The first step in solving this problem is reshaping the dataset so that we can make conclusions about each column rather than each row like in typical machine learning problems.

In [9]:
df = pd.read_csv("train_data.csv")
df.head()

,id,fname,lname,email,gender,ip_address,phone,address,ssn,url,money,credit_card
0,1,Anette,Townley,atownley0@youtube.com,Female,209.246.72.68,741-657-8642,8789 Helena Trail,396-44-3232,http://foxnews.com/ligula/suspendisse.json,$9.17,3564171574160199
1,2,Delmar,Mashal,dmashal1@t.co,Male,169.168.193.114,122-713-3539,7350 Grasskamp Park,823-50-6902,https://imgur.com/et/ultrices.js,$6.09,6333951583101364
2,3,Mikael,Hand,mhand2@toplist.cz,Male,93.100.241.134,790-176-5413,181 Nevada Pass,835-89-4248,http://meetup.com/donec/ut/mauris.jpg,$6.41,3555699034813057
3,4,Jasper,Shurville,jshurville3@typepad.com,Male,18.195.232.182,940-825-3573,8733 Fuller Place,741-15-8957,https://nbcnews.com/montes/nascetur/ridiculus/...,$3.26,5602257897531692348
4,5,Maurizia,Buyers,mbuyers4@bravesites.com,Female,97.247.49.44,497-819-7266,74 8th Parkway,394-89-9716,http://goo.ne.jp/justo.js,$3.97,5164797606744326


To do this we transpose the data and add a new column to this new dataframe that contains the category of data that the row contained.

In [10]:
df = df.transpose()
df['category'] = df.index
df = df.drop('id')
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,category
fname,Anette,Delmar,Mikael,Jasper,Maurizia,Joelle,Ware,Donnie,Travus,Nat,...,Glad,Aguste,Karalynn,Corenda,Lorant,Clair,Percival,Sigismondo,Cullan,fname
lname,Townley,Mashal,Hand,Shurville,Buyers,Shevels,Butchart,Radki,Sperling,Liddicoat,...,Milligan,Fullegar,Shorthouse,O'Driscoll,Gulleford,Pittet,Kneal,Knevet,Minigo,lname
email,atownley0@youtube.com,dmashal1@t.co,mhand2@toplist.cz,jshurville3@typepad.com,mbuyers4@bravesites.com,jshevels5@narod.ru,wbutchart6@virginia.edu,dradki7@multiply.com,tsperling8@amazon.de,nliddicoat9@washingtonpost.com,...,gmilliganrj@patch.com,afullegarrk@1und1.de,kshorthouserl@zimbio.com,codriscollrm@godaddy.com,lgullefordrn@abc.net.au,cpittetro@topsy.com,pknealrp@xing.com,sknevetrq@hhs.gov,cminigorr@fc2.com,email
gender,Female,Male,Male,Male,Female,Female,Male,Female,Male,Female,...,Female,Male,Female,Female,Male,Male,Male,Male,Male,gender
ip_address,209.246.72.68,169.168.193.114,93.100.241.134,18.195.232.182,97.247.49.44,127.5.10.145,253.87.180.132,74.64.60.179,59.204.126.170,187.151.127.144,...,120.23.187.176,131.118.208.233,16.217.219.226,114.68.27.59,16.151.214.226,48.77.73.80,137.200.65.174,248.186.47.65,19.96.101.139,ip_address
phone,741-657-8642,122-713-3539,790-176-5413,940-825-3573,497-819-7266,188-133-3534,600-858-7738,303-788-9360,122-894-3641,194-626-9129,...,984-712-4884,400-811-7576,717-366-6168,795-886-3236,509-934-5178,861-140-4529,999-835-2690,524-582-0084,446-528-2054,phone
address,8789 Helena Trail,7350 Grasskamp Park,181 Nevada Pass,8733 Fuller Place,74 8th Parkway,1190 Morrow Center,9 Ridgeway Street,566 Rusk Road,825 8th Street,97330 Upham Avenue,...,08 Ilene Court,422 Veith Avenue,97375 Delladonna Avenue,7 Marcy Drive,637 Dawn Plaza,0 Clemons Plaza,4 Park Meadow Circle,29905 Talmadge Court,75924 Browning Plaza,address
ssn,396-44-3232,823-50-6902,835-89-4248,741-15-8957,394-89-9716,800-22-6859,734-59-1559,476-02-7834,644-90-9296,138-17-4523,...,853-88-2033,391-75-8040,330-74-9373,811-57-7775,126-91-5152,411-32-0224,631-86-0188,713-55-0607,234-98-9507,ssn
url,http://foxnews.com/ligula/suspendisse.json,https://imgur.com/et/ultrices.js,http://meetup.com/donec/ut/mauris.jpg,https://nbcnews.com/montes/nascetur/ridiculus/...,http://goo.ne.jp/justo.js,https://cocolog-nifty.com/in/purus.html,https://storify.com/nulla/nisl/nunc.html,http://posterous.com/bibendum/felis/sed/interd...,http://uol.com.br/diam/neque/vestibulum/eget/v...,https://thetimes.co.uk/in/faucibus/orci/luctus...,...,http://illinois.edu/sociis/natoque/penatibus/e...,http://ow.ly/eget/eleifend/luctus/ultricies/eu...,http://ed.gov/mauris/sit.xml,https://cafepress.com/felis/ut/at/dolor/quis/o...,http://cloudflare.com/tincidunt/eget/tempus.json,https://microsoft.com/egestas/metus/aenean/fer...,https://wufoo.com/quisque/id/justo/sit/amet/sa...,http://purevolume.com/imperdiet/sapien/urna/pr...,https://vimeo.com/molestie/hendrerit/at/vulput...,url
money,$9.17,$6.09,$6.41,$3.26,$3.97,$3.66,$1.34,$2.74,$7.66,$4.44,...,$6.34,$8.46,$3.70,$6.01,$8.84,$5.67,$0.40,$5.58,$7.11,money


After this, we need to melt the dataframe so that it has a long shape instead of a wide shape. By using the 'category' column that we added earlier as the id variable for the melt, we end up with a new dataframe that has the schema ('category', 'variable', 'value'). The 'variable' variable here is useless to us so we can drop it from the dataframe. Now we have a dataframe that has the schema ('category', 'value').

In [11]:
df = df.melt(id_vars=['category'])
df = df.drop('variable', axis=1)
df.head(10)

,category,value
0,fname,Anette
1,lname,Townley
2,email,atownley0@youtube.com
3,gender,Female
4,ip_address,209.246.72.68
5,phone,741-657-8642
6,address,8789 Helena Trail
7,ssn,396-44-3232
8,url,http://foxnews.com/ligula/suspendisse.json
9,money,$9.17


Using this new dataframe, we have extactly what we need to train our model, but we still need to check for missing/NaN values in the dataset.

<h2><center>ADD SOME CODE FOR DATA CLEANING HERE </center></h2>

# Building the Model #

After cleaning and reshaping our training dataset, we can start building our model. Lets take a quick look at our data.

In [12]:
df.head(10)

,category,value
0,fname,Anette
1,lname,Townley
2,email,atownley0@youtube.com
3,gender,Female
4,ip_address,209.246.72.68
5,phone,741-657-8642
6,address,8789 Helena Trail
7,ssn,396-44-3232
8,url,http://foxnews.com/ligula/suspendisse.json
9,money,$9.17


To build our model, we can't use strings which is all we have in our dataset. So we need to convert these strings into some sort of number that the model will be able to do math with.

Enter scikit-learn's HashingVectorizer. This vectorizer performs a hashing algorithm on the data you pass into it and gives us a sparse matrix of floats that describe each string. By default, the HashingVectorizer does its vectorization assuming there are $ 2^{20} $ features. According to the documentation this is usually a good number of features for text classification.

In [184]:
hash_vect = HashingVectorizer()
jl.dump(hash_vect, 'vectorizer.joblib')

['vectorizer.joblib']

The next step is to extract our dependent and independent variables from the dataframe. Our independent variable(X) will be the 'value' column of our dataframe and our dependent variable(y) will be the 'category' column of the dataframe. However, since our independent variable has to be numerical, we need to use our vectorizer to vectorize the strings.

In [18]:
df['value'] = df['value'].astype(str)

In [19]:
X = hash_vect.fit_transform(df['value'])
y = df['category']

After extracting our variables we can create a test/train split to build our model.

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

Based on scikit-learn's machine learning cheat sheet, we decided that a LinearSVC would most likely be the best model for this problem.

So now we can create the model and train it using our training data.

In [185]:
clf = LinearSVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
jl.dump(clf, 'classifier.joblib')

['classifier.joblib']

Here we can see that our model performs pretty well out of the box, with an accuracy around 75%. However, we haven't done any hyperparameterization yet. In later sprints, we will refine this model using cross-validation/grid search/etc.

## Building Classification Objects ##

When we are given new data, we will perform the same sort of data massaging as we did when building our model. However, we will use the model we built earlier to create a new column in the massaged dataframe. This column will include the 'guess' that our model made for that value.

In [153]:
df = pd.read_csv('test_data.csv')
df_orig = df
df = df.transpose()
df['category'] = df.index
df = df.drop('id').melt(id_vars=['category']).drop('variable', axis=1)
df.head()

,category,value
0,firstname,Giacomo
1,lastname,Pulford
2,email,gpulford0@wsj.com
3,gender,Male
4,ip_address,74.141.110.4


<h4>NOTE: This dataset has different column names for fname and lname</h4>

In [154]:
df['guess'] = clf.predict(hash_vect.fit_transform(df['value']))

In [155]:
df.head(10)

,category,value,guess
0,firstname,Giacomo,lname
1,lastname,Pulford,lname
2,email,gpulford0@wsj.com,email
3,gender,Male,gender
4,ip_address,74.141.110.4,ip_address
5,phone,934-562-0496,phone
6,address,6144 Bonner Pass,address
7,ssn,446-52-7673,money
8,firstname,Virge,fname
9,lastname,Le Grove,lname


Using this new column, we will need to calculate some sort of 'precision' score for each 'category' in the original dataset. Using this precision score, we can determine whether a 'category' belongs to one of the categories that our model can predict. If a category's 'precision' score is too low, then we can consider it as being some sort of new category of data that we haven't encountered yet.

Using this new data frame we can group the guesses together to present our findings to the user.

In [156]:
df.head()

,category,value,guess
0,firstname,Giacomo,lname
1,lastname,Pulford,lname
2,email,gpulford0@wsj.com,email
3,gender,Male,gender
4,ip_address,74.141.110.4,ip_address


We want to build a frequency matrix for 

In [157]:
result = {}
for t in df.itertuples():
    actual = t.category
    guess = t.guess
    
    if actual not in result.keys():
        result[actual] = {}
        
    if guess not in result[actual].keys():
        result[actual][guess] = 1
    else:
        result[actual][guess] = result[actual][guess] + 1

result

{'firstname': {'lname': 853, 'fname': 143, 'address': 1, 'credit_card': 3},
 'lastname': {'lname': 989,
  'fname': 5,
  'email': 4,
  'ssn': 1,
  'credit_card': 1},
 'email': {'email': 1000},
 'gender': {'gender': 1000},
 'ip_address': {'ip_address': 956, 'money': 44},
 'phone': {'phone': 838, 'ssn': 129, 'ip_address': 26, 'lname': 7},
 'address': {'address': 1000},
 'ssn': {'money': 396,
  'ssn': 433,
  'ip_address': 143,
  'phone': 24,
  'fname': 1,
  'lname': 2,
  'email': 1}}

In [158]:
res_df=pd.DataFrame(result).fillna(0)

In [159]:
res_df.apply(max)

firstname      853.0
lastname       989.0
email         1000.0
gender        1000.0
ip_address     956.0
phone          838.0
address       1000.0
ssn            433.0
dtype: float64

In [160]:
res_df

,firstname,lastname,email,gender,ip_address,phone,address,ssn
address,1.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0
credit_card,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
email,0.0,4.0,1000.0,0.0,0.0,0.0,0.0,1.0
fname,143.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0
gender,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0
ip_address,0.0,0.0,0.0,0.0,956.0,26.0,0.0,143.0
lname,853.0,989.0,0.0,0.0,0.0,7.0,0.0,2.0
money,0.0,0.0,0.0,0.0,44.0,0.0,0.0,396.0
phone,0.0,0.0,0.0,0.0,0.0,838.0,0.0,24.0
ssn,0.0,1.0,0.0,0.0,0.0,129.0,0.0,433.0


In [161]:
freq = res_df.apply(max)/res_df.apply(sum)
freq

firstname     0.853
lastname      0.989
email         1.000
gender        1.000
ip_address    0.956
phone         0.838
address       1.000
ssn           0.433
dtype: float64

In [182]:
class Classification:

    def add_column(self, column):
        self.columns.append(column)

    def add_example(self, example):
        self.examples.append(example)

    def to_json(self):
        return f"""\
{{
    "name": "{self.name}",
    "columns": {self.columns},
    "examples": {self.examples}
}}

            """.replace("'",'"')
    
    def __eq__(self, other):
        return isinstance(other, self.__class__) and self.name == other.name
    
    def __str__(self):
        return f"""Name: {self.name}
            Columns: {self.columns}
            Examples: {self.examples}
        """
        
    def __init__(self, name):
        self.name = name
        self.examples = []
        self.columns = []
        
classification_map = {}
for k,v in freq.items():
    if v >= 0.75:
        classification_map[k] = [res_df[k].idxmax()]
    else:
        classification_map[k] = [res_df[k].idxmax(), res_df[k].drop(res_df[k].idxmax()).idxmax()]

classifications = []
for k, v in classification_map.items():
    for c in v:
        classification = Classification(c)
        if classification not in classifications:
            classification.add_column(k)
            for ex in df[df['category'] == k]['value'].head():
                classification.add_example(ex)
            classifications.append(classification)
for cls in classifications:
    print(cls.to_json())


{
    "name": "lname",
    "columns": ["firstname"],
    "examples": ["Giacomo", "Virge", "Fielding", "Zebadiah", "Chrotoem"]
}

            
{
    "name": "email",
    "columns": ["email"],
    "examples": ["gpulford0@wsj.com", "vlegrove1@reference.com", "fbaiyle2@imageshack.us", "zrichardot3@spotify.com", "coleszkiewicz4@vkontakte.ru"]
}

            
{
    "name": "gender",
    "columns": ["gender"],
    "examples": ["Male", "Male", "Male", "Male", "Male"]
}

            
{
    "name": "ip_address",
    "columns": ["ip_address"],
    "examples": ["74.141.110.4", "220.235.6.159", "222.205.231.178", "9.90.228.199", "35.76.198.142"]
}

            
{
    "name": "phone",
    "columns": ["phone"],
    "examples": ["934-562-0496", "736-696-7582", "149-779-8128", "383-641-4571", "215-835-7270"]
}

            
{
    "name": "address",
    "columns": ["address"],
    "examples": ["6144 Bonner Pass", "223 Loftsgordon Plaza", "483 Oneill Place", "99485 Paget Parkway", "2142 Anzinger Plaza"]


In [173]:
res_df['ssn'].drop()

ValueError: Need to specify at least one of 'labels', 'index' or 'columns'